In [141]:
#ML Tlachac, Left on Read, 2022
#https://github.com/mltlachac/UbiComp2022

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import nltk

from datetime import datetime, timedelta
from dateutil import parser
from datetime import timezone

In [142]:
df = pd.read_csv("textLogsDepreST-CAT2021.csv")
df.head()

,id,UnixTimestamp,Direction,Contact
0,HJERN37PE,1578822661204,1,f48f908ec1db8d3e7f86d516ebdab5213f892b7235c421...
1,HJERN37PE,1578822661793,1,f48f908ec1db8d3e7f86d516ebdab5213f892b7235c421...
2,HJERN37PE,1578822663671,1,f48f908ec1db8d3e7f86d516ebdab5213f892b7235c421...
3,HJERN37PE,1578822694000,2,f48f908ec1db8d3e7f86d516ebdab5213f892b7235c421...
4,HJERN37PE,1578822699411,1,f48f908ec1db8d3e7f86d516ebdab5213f892b7235c421...


In [143]:
scoredf = pd.read_csv("surveysDepreST-CAT.csv")
print(scoredf.columns)
scoredf.head()

Index(['id', 'appVersion', 'PHQ - Q1', 'PHQ - Q2', 'PHQ - Q3', 'PHQ - Q4',
       'PHQ - Q5', 'PHQ - Q6', 'PHQ - Q7', 'PHQ - Q8', 'PHQ - Q9',
       'PHQ - Total', 'GAD - Q1', 'GAD - Q2', 'GAD - Q3', 'GAD - Q4',
       'GAD - Q5', 'GAD - Q6', 'GAD - Q7', 'GAD - Total', 'Gender', 'Age',
       'StudentStatus', 'PriorDepressionTreatment', 'Remote', 'COVID', 'Group',
       'Timestamp'],
      dtype='object')


,id,appVersion,PHQ - Q1,PHQ - Q2,PHQ - Q3,PHQ - Q4,PHQ - Q5,PHQ - Q6,PHQ - Q7,PHQ - Q8,...,GAD - Q7,GAD - Total,Gender,Age,StudentStatus,PriorDepressionTreatment,Remote,COVID,Group,Timestamp
0,1MZ0A5PA6,stereotype threat,2,1,2,3,2,1,3,0,...,1,12,Woman,24-39,No & I am not a student,Yes,Yes - Full Time,No\/I don't know,White\/Caucasian,"""2021-03-21 00:29:43.016253"""
1,E7EEQIEOV,control,1,0,1,1,0,1,1,0,...,1,5,Woman,40-55,No & I am not a student,Prefer not to answer,No - Not At All,Yes,White\/Caucasian,"""2021-03-20 00:16:25.943488"""
2,IHNGBZ4G4,control,1,1,2,3,3,2,2,1,...,1,7,Woman,40-55,No & I am not a student,Yes,No - Not At All,No\/I don't know,White\/Caucasian,"""2021-03-21 22:33:41.260353"""
3,R8CI0EGMO,stereotype threat,1,2,3,2,2,3,3,0,...,0,11,Woman,40-55,No & I am not a student,Yes,Yes - Full Time,No\/I don't know,White\/Caucasian,"""2021-03-20 05:03:37.222712"""
4,FWJ1GB40T,control,2,2,2,3,1,2,3,1,...,1,19,Other,18-23,Yes & I am a undergrad student,Yes,No - Not At All,No\/I don't know,White\/Caucasian,"""2021-03-20 11:47:32.838462"""


In [144]:
def unixTS(p):
    sdf = scoredf[scoredf["id"] == p].reset_index()
    s = str(sdf["Timestamp"][0])
    phq = int(sdf['PHQ - Total'][0])
    gad = int(sdf['GAD - Total'][0])
    
    year = int(s[1:5])
    month = int(s[6:8])
    day = int(s[9:11])
    hour = int(s[12:14])
    minute = int(s[15:17])
    second = int(s[18:20])

    dt = datetime(year,month,day,hour,minute,second)
    ts = dt.replace(tzinfo=timezone.utc).timestamp()
    ts = datetime.fromtimestamp(ts)
    return(ts,phq,gad)

#value = unixTS(p)

In [149]:
ndays = 14

print("Number of participants: " + str(len(set(df["id"]))))

participant = []
contacts = []
conversations = []
replies = []
latencylist = []
phqlist = []
gadlist = []
incomingT = []
outgoingT = []
incomingC = []
outgoingC = []

for p in set(df["id"]):
    print(p)
    #limit to participant p
    
    dfp = df[df["id"] == p]
    dfp = dfp.sort_values(by=["UnixTimestamp"])
    
    #get window of ndays
    timeEnd,phq,gad = unixTS(p)
    print(timeEnd)
    print(phq)
    print(gad)
    timeStart = timeEnd - timedelta(days=ndays)
    ts = int(timeStart.replace(tzinfo=timezone.utc).timestamp()*1000)
    
    #limit data to ndays
    dfp = dfp[dfp["UnixTimestamp"]>ts]
    dfin = dfp[dfp["Direction"] == 1].shape[0]
    dfout = dfp[dfp["Direction"] == 2].shape[0]
    
    latencies = []
    ccount = 0
    cin = 0
    cout = 0
    #limiti to contact c
    for c in set(dfp["Contact"]):
        dfc = dfp[dfp["Contact"] == c]
        #check if incoming and outgoing messages
        if len(set(dfc["Direction"])) == 2:
            cin = cin + dfc[dfc["Direction"] == 1].shape[0]
            cout = cout + dfc[dfc["Direction"] == 2].shape[0]
            ccount = ccount + 1
            directions = list(dfc["Direction"])
            times = list(dfc["UnixTimestamp"])
            for i in range(1, len(directions)):
                if (directions[i] == 2) & (directions[i-1] == 1):
                    first = datetime.fromtimestamp(int(times[i-1])/1000)
                    second = datetime.fromtimestamp(int(times[i])/1000)
                    diff = (second-first).seconds
                    latencies.append(diff)
    
    #fill in lists for df
    participant.append(p)
    phqlist.append(phq)
    gadlist.append(gad)
    contacts.append(len(set(dfp["Contact"])))
    conversations.append(ccount)
    replies.append(len(latencies))
    latencylist.append(latencies)
    incomingT.append(dfin)
    outgoingT.append(dfout)
    incomingC.append(cin)
    outgoingC.append(cout)

Number of participants: 348
UM8C3DPOI
2021-04-06 17:47:10
18
11
7KBUI5ZGZ
2021-03-21 23:25:14
14
14
QUWE365TI
2021-03-11 11:04:53
7
2
0832IIHPX
2021-03-13 17:03:41
7
6
C1BOB71XJ
2021-03-28 23:15:41
9
10
R7TFC1NBN
2021-03-13 14:46:27
11
20
Q6N7PT68V
2021-03-13 13:49:18
12
8
QBEZBGLGH
2020-12-24 10:10:44
15
7
C4NCER7RE
2021-03-30 18:01:22
6
2
7MXTEUBZM
2021-03-13 14:00:16
11
8
SKNKBM56D
2021-02-16 06:52:39
14
10
7A9PXD4HB
2021-04-02 06:33:03
8
13
WEI4XXWKK
2021-03-02 18:55:12
9
5
AJFHFELJ9
2021-03-31 07:54:26
2
3
PG76MAUMX
2021-04-07 22:10:45
6
1
SGVN7E3SM
2021-03-17 00:16:20
0
0
WX00908TW
2021-03-31 04:11:49
5
4
XT11CP7OL
2021-04-09 18:40:25
18
14
GUN9FOB9Z
2021-03-09 09:22:07
13
9
2W1EYGWM0
2021-03-20 09:27:17
22
17
IUGAHUB8Z
2021-03-02 19:30:28
12
13
WB0TLP5X4
2021-03-02 06:40:10
10
12
H9OW9LNOC
2021-03-09 09:29:20
26
16
0U9XV75IX
2021-03-06 15:49:05
16
10
XRN7RMADR
2021-03-25 18:39:41
7
3
5BPFFMIFV
2021-04-14 17:35:08
12
2
9UK1BSBLV
2021-03-13 15:26:04
6
1
XDQVFTSQ9
2021-03-02 17:42:

2021-03-31 02:47:39
4
1
ES9H14HA2
2021-03-21 14:30:48
24
19
YS0ZSCTRI
2021-03-18 13:06:01
4
4
C0FO8A3YF
2021-03-13 15:25:43
0
0
QT3M2ARB7
2021-03-25 21:26:28
1
0
FWBF74Q1R
2021-03-02 21:05:51
9
3
ZE5QAUQQL
2021-03-13 17:11:40
20
12
RO36LNUYG
2021-03-25 17:37:58
11
9
8J0PZ4W9Y
2021-04-05 18:44:41
14
17
DDIBMJW86
2021-03-02 16:55:41
11
13
87I1YSTNQ
2021-03-02 07:18:03
13
8
GZU3KNP1E
2021-03-25 18:35:01
0
0
FANIYNL6M
2021-03-13 17:32:54
11
17
PGU3O10D9
2021-03-13 16:46:04
22
18
5XEUPOVMZ
2021-02-05 00:05:08
26
21
5AH5AX23X
2021-03-13 16:25:23
20
15
VN88VQWFH
2021-03-02 21:34:51
5
2
8CLVULOIO
2021-04-05 15:20:50
5
9
C9RIGOBMB
2021-03-24 13:47:39
27
18
XOL6TDL8W
2021-03-31 04:55:38
0
0
0GJVGLPFD
2021-03-02 20:34:16
6
1
OVW13SMGC
2021-03-26 09:36:58
3
5
AP4FPZP5H
2021-03-27 17:26:24
27
21
SMT4HMHV3
2021-03-02 18:35:52
15
8
AZTQ43FR2
2021-03-26 19:23:48
5
2
I2QVYX8KA
2021-02-16 06:22:30
12
6
80HSQUGOP
2021-03-02 08:14:15
17
10
IA81BBK3O
2021-04-05 21:26:04
5
5
3CHEWWWZ3
2021-03-17 22:13:28
16

In [154]:
#create df with latencies
newdf = pd.DataFrame()
newdf['id'] = participant
newdf["phq"] = phqlist
newdf["gad"] = gadlist
newdf['latency'] = latencylist
newdf['contacts'] = contacts
newdf['incomingTotal'] = incomingT
newdf['outgoingTotal'] = outgoingT
newdf['conversations'] = conversations
newdf['incomingTexts'] = incomingC
newdf['outgoingTexts'] = outgoingC
newdf['replies'] = replies
newdf.head()

,id,phq,gad,latency,contacts,incomingTotal,outgoingTotal,conversations,incomingTexts,outgoingTexts,replies
0,UM8C3DPOI,18,11,[],27,96,0,0,0,0,0
1,7KBUI5ZGZ,14,14,"[30434, 2149, 19, 12700, 208, 1803, 287, 138, ...",21,127,84,3,64,56,41
2,QUWE365TI,7,2,[],13,23,0,0,0,0,0
3,0832IIHPX,7,6,[],33,372,0,0,0,0,0
4,C1BOB71XJ,9,10,[],11,153,0,0,0,0,0


In [155]:
print(newdf.shape[0])
newdf = newdf[newdf.conversations > 0]
print(newdf.shape[0])
newdf = newdf[newdf.replies > 0]
print(newdf.shape[0])
newdf = newdf[newdf.replies > 1]
print(newdf.shape[0])

348
88
85
69


In [156]:
#extract latency features
newdf = newdf.reset_index()

responses = []
minlist = []
quant10 = []
quant25 = []
quant50 = []
quant75 = []
quant90 = []
maxlist = []
median = []

for i in range(0, newdf.shape[0]):
    responses.append(len(newdf.latency[i]))
    minlist.append(min(newdf.latency[i]))
    quant10.append(np.quantile(newdf.latency[i], 0.1))
    quant25.append(np.quantile(newdf.latency[i], 0.25))
    quant50.append(np.quantile(newdf.latency[i], 0.5))
    quant75.append(np.quantile(newdf.latency[i], 0.75))
    quant90.append(np.quantile(newdf.latency[i], 0.9))
    maxlist.append(max(newdf.latency[i]))
    median.append(np.mean(newdf.latency[i]))
    
newdf['responses'] = responses
newdf['min'] = minlist
newdf['quant10'] = quant10
newdf['quant25'] = quant25
newdf['quant50'] = quant50
newdf['quant75'] = quant75
newdf['quant90'] = quant90
newdf['max'] = maxlist
newdf['mean'] = median

In [157]:
newdf.to_csv("LatencyTexts" + str(ndays) + "CAT.csv")